In [167]:
# Install required libraries
%pip install nltk sacrebleu

Note: you may need to restart the kernel to use updated packages.


In [166]:
import math
import sacrebleu
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nsadi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [152]:
# Example data
reference_sentence = "she was interested in world history because she read the book."
hypothesis_sentence = "she read the book because she was interested in world history."

1. Formality: This metric is calculated by evaluating a set of transformed texts against standard criteria for formality and calculating the percentage of texts that meet these criteria. The calculation could be represented as:


In [188]:
# # def calculate_formality(hypothesis: str, reference: str):
# #     # formal_texts = [text for text in transformed_texts if meets_criteria(text, standard_criteria)]
# #     # formality_score = len(formal_texts) / len(transformed_texts)

# #     return formality_score

# from transformers import BertTokenizer, BertForSequenceClassification
# import torch

# # Load pre-trained BERT model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model.eval()

# def your_formality_classifier(sentence: str) -> float:
#     """
#     Advanced implementation of a formality classifier using BERT.
#     Replace this with your actual implementation.

#     Parameters:
#     - sentence (str): The input sentence to classify.

#     Returns:
#     - formality_score (float): The formality score for the sentence.
#     """
#     # Tokenize and convert to model input format
#     inputs = tokenizer(sentence, return_tensors="pt")
    
#     # Forward pass through the model
#     with torch.no_grad():
#         outputs = model(**inputs)

#     # Extract logits and apply softmax to get probabilities
#     logits = outputs.logits
#     probabilities = torch.nn.functional.softmax(logits, dim=1)

#     # Assuming the model is trained for binary classification (formal vs. informal)
#     # You may need to adapt this based on your specific classification setup
#     formal_probability = probabilities[0, 1].item()

#     # You may need to fine-tune these thresholds based on your data
#     formality_threshold_high = 0.6
#     formality_threshold_low = 0.4

#     print(formal_probability)

#     # Classify based on probability thresholds
#     # if formal_probability > formality_threshold_high:
#     #     formality_score = 1.0  # More formal
#     # elif formal_probability < formality_threshold_low:
#     #     formality_score = -1.0  # Less formal
#     # else:
#     #     formality_score = 0.0  # Neutral or in-between

#     return formal_probability

# # Example usage:
# hypothesis_sentence = "you so suppa"
# reference_sentence = "I appreciate the image"

# formality_score = your_formality_classifier(hypothesis_sentence)
# print(f"Formality Score: {formality_score}")


2. Bilingual Evaluation Understudy (BLEU): BLEU score is calculated by measuring the overlap of n-grams between the candidate translation and the references. The calculation could be represented as:

In [50]:
#Higher is better
def calculate_bleu(hypothesis: str, reference: str):
    hypothesis_split = hypothesis.split()
    reference_split = reference.split()

    bleu_score = sentence_bleu([reference_split], hypothesis_split)
   
    return bleu_score


In [165]:
bleu_score = calculate_bleu(hypothesis_sentence, reference_sentence)

print(f"BLEU Score: {bleu_score:.3f}")

BLEU Score: 0.000


3. Translation Edit Rate Plus (TERp): TERp is calculated by measuring the matching flaw between machine-generated translations and human-created translation. The calculation could be represented as:

In [157]:
#Higher is better
def calculate_terp(hypothesis: str, reference: str, phrase_table, edit_costs):
    hypothesis_tokens = hypothesis.split()
    reference_tokens = reference.split()

    # TERp by Stem Matches, Synonym Matches, and Phrase Substitutions
    stem_matches = calculate_stem_matches(hypothesis_tokens, reference_tokens)
    synonym_matches = calculate_synonym_matches(hypothesis_tokens, reference_tokens)
    phrase_substitutions = calculate_phrase_substitutions(hypothesis_tokens, reference_tokens, phrase_table, edit_costs)

    # Calculate TERp score
    terp_score = (stem_matches + synonym_matches + phrase_substitutions) / len(reference_tokens)

    return terp_score

def calculate_stem_matches(hypothesis_tokens, reference_tokens):
    stemmer = PorterStemmer()
    stem_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                      if stemmer.stem(hyp_token.lower()) == stemmer.stem(ref_token.lower()))
    return stem_matches

def calculate_synonym_matches(hypothesis_tokens, reference_tokens):
    synonym_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                          if are_synonyms(hyp_token.lower(), ref_token.lower()))
    return synonym_matches

def are_synonyms(word1, word2):
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)
    
    return any(set1.wup_similarity(set2) > 0.7 for set1 in synsets1 for set2 in synsets2)

def calculate_phrase_substitutions(hypothesis_tokens, reference_tokens, phrase_table, edit_costs):
    substitution_cost = 0

    for i in range(len(hypothesis_tokens)):
        for j in range(len(reference_tokens)):
            if (hypothesis_tokens[i], reference_tokens[j]) in phrase_table:
                # Retrieve paraphrase information from the phrase table
                paraphrase_info = phrase_table[(hypothesis_tokens[i], reference_tokens[j])]
                
                # Calculate the cost using the provided formula
                cost = (
                    edit_costs['w1'] +
                    edit_costs['w2'] * paraphrase_info['edit'] * math.log(paraphrase_info['probability']) +
                    edit_costs['w3'] * paraphrase_info['edit'] * paraphrase_info['probability'] +
                    edit_costs['w4'] * paraphrase_info['edit']
                )

                # Ensure the substitution cost is not negative
                substitution_cost += max(0, cost)

    return substitution_cost

def terp_alignment(hypothesis, reference, phrase_table, edit_costs):
    alignment = []

    for hyp_token, ref_token in zip(hypothesis.split(), reference.split()):
        if hyp_token == ref_token:
            alignment.append((hyp_token, ref_token, "Exact Match"))
        else:
            alignment.append((hyp_token, ref_token, "Mismatch"))

    return alignment

In [162]:
# Example usage:
hypothesis_sentence = "This is an example sentence."
reference_sentence = "This is a instance sentence."

# Calculate TERp score
terp_score = calculate_terp(hypothesis_sentence, reference_sentence)
print(f"TERp Score: {terp_score}")

# Generate alignment
alignment = terp_alignment(hypothesis_sentence, reference_sentence)
print("Alignment:", alignment)

TERp Score: 1.0
Alignment: [('This', 'This', 'Exact Match'), ('is', 'is', 'Exact Match'), ('an', 'a', 'Mismatch'), ('example', 'instance', 'Mismatch'), ('sentence.', 'sentence.', 'Exact Match')]


In [116]:
# Lower is better
def calculate_ter(hypothesis, reference):
    # Tokenize the input sentences into lists of words
    hypothesis_tokens = hypothesis.split()
    reference_tokens = reference.split()

    # Compute Levenshtein distance between hypothesis and reference
    distance = levenshtein_distance(hypothesis_tokens, reference_tokens)

    # Compute TER score
    ter_score = distance / len(reference_tokens)
    
    return ter_score

def levenshtein_distance(s1, s2):
    # Initialize a matrix to store the distances
    matrix = [[0] * (len(s2) + 1) for _ in range(len(s1) + 1)]

    # Initialize the first row and column
    for i in range(len(s1) + 1):
        matrix[i][0] = i
    for j in range(len(s2) + 1):
        matrix[0][j] = j

    # Fill in the matrix
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            matrix[i][j] = min(
                matrix[i - 1][j] + 1,  # Deletion
                matrix[i][j - 1] + 1,  # Insertion
                matrix[i - 1][j - 1] + cost  # Substitution
            )

    # Return the final edit distance
    return matrix[len(s1)][len(s2)]


In [114]:
ter_score = calculate_ter(hypothesis_sentence, reference_sentence)

print(f"TER Score: {ter_score}")

TER Score: 0.25


4. Paraphrase In N-gram Changes (PINC): PINC is calculated by computing the percentage of n-grams that appear in the candidate sentence but not in the source sentence. The calculation could be represented as:

In [85]:
# Lower is better
def calculate_pinc(hypothesis: str, reference: str, n: int):
    hypothesis_split = hypothesis.split()
    reference_split = reference.split()

    hypothesis_ngrams = set(ngrams(hypothesis_split, n))
    reference_ngrams = set(ngrams(reference_split, n))
    new_ngrams = hypothesis_ngrams - reference_ngrams
    pinc_score = len(new_ngrams) / len(hypothesis_ngrams)

    return pinc_score

PINC Score: 0.6666666666666666


In [93]:
pinc_score = calculate_pinc(hypothesis_sentence, reference_sentence, 2)

print(f"PINC Score: {pinc_score}")

PINC Score: 0.0
